In [1]:
%matplotlib inline

In [2]:
import theano

In [3]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 18.877000 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu


In [4]:
import os
import sys
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import keras
#from keras import backend as K
from keras.callbacks import LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using Theano backend.


In [5]:
cwd = os.getcwd()
cwd

'C:\\Users\\Chris\\Documents\\kaggle\\personalized medicine\\misc-master\\misc-master'

In [6]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is AC1A-A21A

 Directory of C:\Users\Chris\Documents\kaggle\personalized medicine\misc-master\misc-master

09/30/2017  08:31 PM    <DIR>          .
09/30/2017  08:31 PM    <DIR>          ..
09/30/2017  08:31 PM    <DIR>          .ipynb_checkpoints
09/30/2017  08:31 PM            62,607 full_prec_med_text_vars_API-20170929.ipynb
09/30/2017  08:27 PM    <DIR>          out
09/30/2017  08:25 PM                 6 README.md
09/30/2017  08:25 PM            19,675 stage2_5eps_labels.csv
09/30/2017  08:25 PM           144,574 stage2_preds.csv
09/30/2017  08:25 PM           152,976 stage2_preds_4epochs.csv
09/30/2017  08:25 PM           153,967 stage2_preds_5epochs.csv
               6 File(s)        533,805 bytes
               4 Dir(s)  423,226,572,800 bytes free


In [7]:
TEST_TEXT_DIR = '/home/rhooads/kaggle/'
TEST_VARIANTS_DIR = '/home/rhooads/kaggle/'
STAGE1_SOLUTIONS_DIR = '/home/rhooads/kaggle/'

TRAINING_TEXT_DIR = '/home/rhooads/kaggle/'
TRAINING_VARIANTS_DIR = '/home/rhooads/kaggle/'

STAGE2_TEXT_DIR = '/home/rhooads/kaggle/'
STAGE2_VARIANTS_DIR = '/home/rhooads/kaggle/'

GLOVE_DIR = '/home/rhooads/kaggle/'

In [8]:
test_text = pd.read_csv(os.path.join(TEST_TEXT_DIR, 'test_text'), sep = '\n')
test_text.head()

IOError: File /home/rhooads/kaggle/test_text does not exist

In [ ]:
test_text = test_text['ID,Text'].str.split('\|\|', expand=True, n=1)
test_text = test_text.rename(columns={0: "ID", 1: "Text"})
test_text['ID'] = test_text['ID'].astype(int)

In [ ]:
test_text.tail()
#test_text.shape
type(test_text['ID'].values[0])

In [ ]:
test_variants = pd.read_csv(os.path.join(TEST_VARIANTS_DIR, 'test_variants'))
#train_vars = train_vars.sort_index(axis=1)
test_variants['Group'] = 'test'

In [ ]:
test_variants.tail()
#test_variants.shape

In [ ]:
stage1_solutions = pd.read_csv(os.path.join(STAGE1_SOLUTIONS_DIR, 'stage1_solution_filtered.csv'))
stage1_solutions.head()
#stage1_preclasses = stage1_solutions.drop('ID', axis=1).as_matrix()
#stage1_preclasses
# = pd.DataFrame(stage1_preclasses)
#stage1_preclasses_df.head()

In [ ]:
hist = stage1_solutions.drop('ID', axis=1)
hist = np.argmax(hist.values, axis=1)
#test_labels = pd.DataFrame(test_labels, columns = ['Class'])
#test_labels['Class'].unique()
#test_labels = test_labels.append(pd.DataFrame([2,7,8], columns = ['Class']))
#test_labels.head()
hist

In [ ]:
sns.distplot(hist, bins=20)

In [ ]:
training_text = pd.read_csv(os.path.join(TRAINING_TEXT_DIR, 'training_text'), sep = '\n')
training_text.head()

In [ ]:
training_text = training_text['ID,Text'].str.split('\|\|', expand=True, n=1)
training_text = training_text.rename(columns={0: "ID", 1: "Text"})
training_text['ID'] = training_text['ID'].astype(int)

In [ ]:
training_text.head()
#training_text.shape
type(training_text['ID'].values[0])

In [ ]:
training_variants = pd.read_csv(os.path.join(TRAINING_VARIANTS_DIR, 'training_variants'))
training_variants['Group'] = 'train'

In [ ]:
training_variants.head()
#training_variants.shape

In [ ]:
#import stage2 here to use variants for one hot encoding
stage2_text = pd.read_csv(os.path.join(STAGE2_TEXT_DIR, 'stage2_test_text.csv'), sep = '\n')
stage2_text.head()

In [ ]:
stage2_text = stage2_text['ID,Text'].str.split('\|\|', expand=True, n=1)
stage2_text = stage2_text.rename(columns={0: "ID", 1: "Text"})
stage2_text['ID'] = stage2_text['ID'].astype(int)

In [ ]:
stage2_text.tail()
stage2_text.shape
type(stage2_text['ID'].values[0])

In [ ]:
stage2_variants = pd.read_csv(os.path.join(STAGE2_VARIANTS_DIR, 'stage2_test_variants.csv'))
stage2_variants['Group'] = 'stage2'

In [ ]:
stage2_variants.tail()
#test_variants.shape

In [ ]:
variants=pd.concat([training_variants.drop('Class', axis=1), test_variants, stage2_variants])
variants.head()

In [ ]:
variants.shape

In [ ]:
variant_dummies = pd.get_dummies(variants, columns=['Gene', 'Variation'], drop_first=True)
variant_dummies.head()

In [ ]:
test_variants_dummies = variant_dummies[variant_dummies['Group'] == 'test']
test_variants_dummies.head()

In [ ]:
test_no_labels = test_variants_dummies.merge(test_text, left_on='ID', right_on='ID')
test_union = pd.merge(test_no_labels, stage1_solutions, on='ID', how='left')
test_union.head()

In [ ]:
test_full = test_union[test_union.isnull().any(axis=1)]
test_full = test_full.reset_index(drop=True)
test_full = test_full.iloc[:, :test_full.shape[1]-9]
test_full.head()

In [ ]:
test_full.isnull().values.any()

In [ ]:
test_full.shape

In [ ]:
test_validations = test_union.dropna().copy()
test_validations = test_validations.reset_index(drop=True)
test_validations.head()

In [ ]:
test_validations.shape

In [ ]:
training_variants_dummies = variant_dummies[variant_dummies['Group'] == 'train']
training_variants_dummies.head()

In [ ]:
training_dummies_text = training_variants_dummies.merge(training_text, left_on='ID', right_on='ID')
training_dummies_text.head()

In [ ]:
training_variants['Class'] = training_variants['Class']-1
training_variants.head()

In [ ]:
training_one_hot = keras.utils.to_categorical(training_variants['Class'])
training_one_hot

In [ ]:
training_one_hot_colnames = pd.DataFrame(training_one_hot, columns=['Class_0', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8'])
training_one_hot_colnames.head()

In [ ]:
training_one_hot_colnames.shape

In [ ]:
training_full = pd.concat([training_dummies_text, training_one_hot_colnames], axis=1)
training_full.head()

In [ ]:
training_full.isnull().values.any()

In [ ]:
training_full.shape

In [ ]:
stage2_variants_dummies = variant_dummies[variant_dummies['Group'] == 'stage2']
stage2_variants_dummies.head()

In [ ]:
stage2_full = stage2_variants_dummies.merge(stage2_text, left_on='ID', right_on='ID')
stage2_full.head()

In [ ]:
stage2_full.isnull().values.any()

In [ ]:
stage2_full.shape

In [ ]:
def process_input_text(x):
    x['Text'] = x['Text'].str.replace(r'(Go to: [0-9]. )(?!introduction/?|background).*', r'\1', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'(Go to:) (?!introduction/?|background).*', r'\1', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace('^.*?Go to: [0-9]. (introduction/?|background)', r'\1', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace('^.*?Go to: (introduction/?|background)', r'\1', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'^introduction', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'^background', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'Go to:', '')
    x['Text'] = x['Text'].str.replace('^.*?(Key Words:)', r'\1', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(' Results.*$| RESULTS .*$', '')
    x['Text'] = x['Text'].str.replace('MATERIALS? AND.*$| Methodology.*$', '')
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x98', '\'', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x99', '\'', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x9a', ',', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x9c', '"', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x9d', '"', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'"', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\xa2', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x93', '-', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x94', '-', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x88\xbc', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x8a', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x82', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x80\x83', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x82\xac', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xc2\xae', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xe2\x88\x92', '-', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xc3\x82', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xc2\xa1', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xc3\xa2', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xc2\xa9', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xb1', 'alpha', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xb2', 'beta', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xb3', 'gamma', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xb4', 'delta', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xb5', 'epsilon', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\xce\xba', 'kappa', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\t[0-9]{0,3}\/[0-9]{0,3}', '', flags=re.IGNORECASE) #reference
    x['Text'] = x['Text'].str.replace(r'\t\[.*?\]', '', flags=re.IGNORECASE) #reference
    x['Text'] = x['Text'].str.replace(r'\((\d+, ?)+(\d+)?\)', '', flags=re.IGNORECASE) #comma sep ref list in rd brackets
    x['Text'] = x['Text'].str.replace(r'\(\d+\)', '', flags=re.IGNORECASE) #single reference in rd brack
    x['Text'] = x['Text'].str.replace(r'\(\d+-\d+\)', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\[\d+\]', '', flags=re.IGNORECASE) #reference ref in sq brack
    x['Text'] = x['Text'].str.replace(r'\[\d+-\d+\]', '', flags=re.IGNORECASE) #reference x-y in sq brack
    x['Text'] = x['Text'].str.replace(r'\[(\d+, ?)+(\d+)?\]', '', flags=re.IGNORECASE) #comma sep ref list in sq brackets
    x['Text'] = x['Text'].str.replace(r'\(\w+? et al.*?\)', '', flags=re.IGNORECASE) #et el.
    x['Text'] = x['Text'].str.replace(r'([A-Za-z]{6,}?)[0-9]+?(\.)', r'\1\2', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'([A-Za-z]{6,}?)[0-9]+?(,)', r'\1\2', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'([A-Za-z]{6,}?)(\d+, ?)+(\d+)?(\.)', r'\1\4', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'(\d{1,3}?, ?)+(\d+)?(\.)', r'\3', flags=re.IGNORECASE) #open csl
    x['Text'] = x['Text'].str.replace(r'\)(\d{1,3}?)(\.)', r'\2', flags=re.IGNORECASE) #parentheses followed by 1 ref preiod
    x['Text'] = x['Text'].str.replace(r'@\.?EGFR', r'deltaEGFR', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\(to [A-Z]\. [A-Z]\.\)', r'', flags=re.IGNORECASE) #funding abr
    x['Text'] = x['Text'].str.replace(r' \d{4,}\.? ', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\b[tcga]+\b', ' ')
    x['Text'] = x['Text'].str.replace(r'\(Fig\.? ?\d+ ?[A-Za-z]?.{0,4}\)', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\(Figure\.? ?\d+ ?[A-Za-z]?.{0,4}\)', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\d{1,2}\/\d{1,2}\/\d{4}', ' ')
    x['Text'] = x['Text'].str.replace(r'\d{1,2}\/\d{1,2}\/\d{2}', ' ')
    x['Text'] = x['Text'].str.replace(r'((\(\d{3}\) ?)|(\d{3}-))?\d{3}-\d{4}', ' ', flags=re.IGNORECASE) #phone
    x['Text'] = x['Text'].str.replace(r'\(?(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+\/?\)?', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'\(?www\.[\w\-_]+(\.[\w\-_]+)+\/?\)?', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'https?:\/\/(dx\.)?doi\.org\/.*? ', '', flags=re.IGNORECASE) #http://dx?doi.org
    x['Text'] = x['Text'].str.replace(r'doi: ?\d+\.\d+\/.*? ', '', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'Dept\.?|Department of \w+', ' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'Phone:', r' ', flags=re.IGNORECASE)
    x['Text'] = x['Text'].str.replace(r'Fax:', r' ', flags=re.IGNORECASE)
    return (x)

In [ ]:
tt_train = process_input_text(training_full)


In [ ]:
tt_train[0:2]

In [ ]:
tt_train.shape[0]

In [ ]:
for i in range(tt_train.shape[0]):
    tt_train.loc[i,'Text'] = tt_train.loc[i,'Text'].decode("utf-8").encode("ascii","ignore")

In [ ]:
tt_test = process_input_text(test_full)

In [ ]:
tt_test.shape[0]

In [ ]:
for i in range(tt_test.shape[0]):
    tt_test.loc[i,'Text'] = tt_test.loc[i,'Text'].decode("utf-8").encode("ascii","ignore")

In [ ]:
tt_test_valids = process_input_text(test_validations)

In [ ]:
tt_test_valids.shape[0]

In [ ]:
for i in range(tt_test_valids.shape[0]):
    tt_test_valids.loc[i,'Text'] = tt_test_valids.loc[i,'Text'].decode("utf-8").encode("ascii","ignore")

In [ ]:
tt_stage2 = process_input_text(stage2_full)

In [ ]:
tt_stage2[0:2]

In [ ]:
tt_stage2.shape[0]

In [ ]:
for i in range(tt_stage2.shape[0]):
    tt_stage2.loc[i,'Text'] = tt_stage2.loc[i,'Text'].decode("utf-8").encode("ascii","ignore")

In [ ]:
tt_stage2['Text'][150]

In [ ]:
#tt_test['Text'].str.contains('\\\\x')
tt_test.loc[150,'Text']
#asciidata
#data

In [ ]:
tt_test.loc[741]
tt_test.loc[741, 'Text'][:1000]

In [ ]:
#does entry lack "reintroduction
#reint_mask = ~tt['Text'].str.contains('reintroduction', flags=re.IGNORECASE)
#reint_mask[:5]

In [ ]:
#theint_mask = ~tt['Text'].str.contains('the introduction', flags=re.IGNORECASE)
#theint_mask[:5]

In [ ]:
#anint_mask = ~tt['Text'].str.contains('an introduction', flags=re.IGNORECASE)
#anint_mask[:5]

In [ ]:
#theint_mask[742]

In [ ]:
#leng = int(reint_mask.shape[0])
#for i in range(leng):
#    if (reint_mask[i] == True or theint_mask[i] == True or anint_mask[i] == True):
#        tt['Text'][i] = re.sub('Introduction.{,20}?\\xe2\\x80\\xa2.*$', '', tt['Text'][i], flags=re.IGNORECASE)

In [ ]:
#leng = int(reint_mask.shape[0])
#for i in range(leng):
#    if (reint_mask[i] == True or theint_mask[i] == True or anint_mask[i] == True):
#        tt['Text'][i] = re.sub('^.*?Introduction', '', tt['Text'][i], flags=re.IGNORECASE)

In [ ]:
#tt.where(df_mask, tt['Text'].str.replace('Introduction.{,20}?\\xe2\\x80\\xa2.*$', ''))
#tt['Text'].head()
#tt['Text'] = tt['Text'].str.replace('Introduction.{,20}?\\xe2\\x80\\xa2.*$', '')

In [ ]:
#4808 characters - must convert to words for vocab length
len(tt_test.loc[741, 'Text'])

In [ ]:
tt_test.loc[3227]
tt_test.loc[3227, 'Text'][:1000]

In [ ]:
tt_test.loc[923, 'Text']
#tt_test.loc[923, 'Text']

In [ ]:
tt_test.loc[150]
tt_test.loc[150, 'Text']
#1613
#529
#150
#2364
#2395

In [ ]:
sns.distplot(tt_stage2['Text'].str.len())

In [ ]:
###########################
###########################
#RE test here
#
###########################

In [ ]:
tt_stage2['Text'].str.len().sort_values(ascending = True)[:20]
#[50:100]

In [ ]:
lens = pd.DataFrame(tt_test['Text'].str.len())
#lens.head()
#lens.idxmax()
lens.median()
#lens.quantile(q=0.75)
#If needed, just take the first 5000 words in each entry

In [ ]:
train_corpus = tt_train['Text'].values.tolist()
test_corpus = tt_test['Text'].values.tolist()
stage2_corpus = tt_stage2['Text'].values.tolist()
train_test_stage2_corpus = train_corpus + test_corpus + stage2_corpus

test_val_corpus = tt_test_valids['Text'].values.tolist()

In [ ]:
train_corpus[:2]

In [ ]:
test_corpus[:2]

In [ ]:
stage2_corpus[:2]

In [ ]:
train_test_stage2_corpus[:2]

In [ ]:
#IN GENERAL, DONT MAKE OWN VOCAB
#DOWNLAOD GLOVE OR WORD2VEC
SEQ_LEN = 2000
MAX_WORDS = 5000
tokenizer = Tokenizer(num_words=MAX_WORDS)

#or take all words as tokens
#tokenizermax_words = Tokenizer()

In [ ]:
#probably have to use a loop for this
tokenizer.fit_on_texts(train_test_stage2_corpus)
WORD_INDEX = tokenizer.word_index
print(WORD_INDEX)

In [ ]:
import operator
sorted(WORD_INDEX.items(), key=operator.itemgetter(1))
#sorted(WORD_INDEX.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
print('Found %s unique tokens.' % len(WORD_INDEX))

In [ ]:
train_seq = tokenizer.texts_to_sequences(train_corpus)
test_seq = tokenizer.texts_to_sequences(test_corpus)
test_val_seq = tokenizer.texts_to_sequences(test_val_corpus)
stage2_seq = tokenizer.texts_to_sequences(stage2_corpus)

In [ ]:
test_val_seq[0] #1277:1279

In [ ]:
def limit_words(x):
    for i in range(len(x)):
        x[i] = x[i][:SEQ_LEN]
    return (x)

In [ ]:
train_seq_limit = limit_words(train_seq)
test_seq_limit = limit_words(test_seq)
test_val_seq_limit = limit_words(test_val_seq)
stage2_seq_limit = limit_words(stage2_seq)

In [ ]:
len(test_seq[1278])
len(test_seq_limit[1278])
#test_seq_limit[1278]

In [ ]:
train_data = pad_sequences(train_seq_limit, maxlen=SEQ_LEN, value=0)

In [ ]:
test_data = pad_sequences(test_seq_limit, maxlen=SEQ_LEN, value=0)

In [ ]:
test_val_data = pad_sequences(test_val_seq, maxlen=SEQ_LEN, value=0)

In [ ]:
stage2_data = pad_sequences(stage2_seq_limit, maxlen=SEQ_LEN, value=0)

In [ ]:
stage2_data[0]

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
test_val_data.shape

In [ ]:
stage2_data.shape

In [ ]:
EMBEDDING_DIM = 100

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embeddings_index

In [ ]:
embedding_matrix = np.zeros((len(WORD_INDEX) + 1, EMBEDDING_DIM))
for word, i in WORD_INDEX.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix[2:4]

In [ ]:
#########################################3
#######################################333
########################################3
#######################################

#Need to combine training_full and test full genes and vars before one hot encoding - for mlp

In [ ]:
#Come back to this if needed
#pre_MLP_CNN_variants = pd.concat([train_no_text, pd.DataFrame(train_data), pd.DataFrame(train_labels)], axis=1)

In [ ]:
searchfor = ['ID', 'Gene', 'Variation']
searchfor

In [ ]:
train_no_text = training_full.loc[:,training_full.columns.str.contains('|'.join(searchfor))]
train_no_text.head()

In [ ]:
train_no_text.shape

In [ ]:
train_labels = training_full.loc[:,training_full.columns.str.contains('Class')]
train_labels.head()

In [ ]:
train_MLP_CNN = pd.concat([train_no_text, pd.DataFrame(train_data), train_labels], axis=1)
train_MLP_CNN.head()

In [ ]:
#x_split = .iloc[:,train_no_text.shape[1]:]
#x_split = train_MLP_CNN.drop(['Class'], axis=1)
x_split = train_MLP_CNN.iloc[:,:(train_MLP_CNN.shape[1]-9)]
y_split = train_MLP_CNN.iloc[:,(train_MLP_CNN.shape[1]-9):]
x_split.head()
#y_split.head()

In [ ]:
test_val_no_text = test_validations.loc[:,test_validations.columns.str.contains('|'.join(searchfor))]
test_val_no_text.head()

In [ ]:
test_val_no_text.shape

In [ ]:
test_vals_labels = test_validations.iloc[:,(test_validations.shape[1]-9):]
test_vals_labels.head()

In [ ]:
val_MLP_CNN = pd.concat([test_val_no_text, pd.DataFrame(test_val_data), test_vals_labels], axis=1)
val_MLP_CNN.head()

In [ ]:
val_MLP_CNN.shape

In [ ]:
x_validation = val_MLP_CNN.iloc[:,:(val_MLP_CNN.shape[1]-9)]
y_validation = val_MLP_CNN.iloc[:,(val_MLP_CNN.shape[1]-9):]
x_validation.head()
#y_validation.head()

In [ ]:
test_full_no_text = test_full.loc[:,test_full.columns.str.contains('|'.join(searchfor))]
test_full_no_text.head()

In [ ]:
test_full_no_text.shape

In [ ]:
test_MLP_CNN = pd.concat([test_full_no_text, pd.DataFrame(test_data)], axis=1)
test_MLP_CNN.head()

In [ ]:
test_MLP_CNN.shape

In [ ]:
stage2_no_text = stage2_full.loc[:,stage2_full.columns.str.contains('|'.join(searchfor))]
stage2_no_text.head()

In [ ]:
stage2_no_text.head()
stage2_no_text.shape[1]
stage2_no_text.shape

In [ ]:
stage2_MLP_CNN = pd.concat([stage2_no_text, pd.DataFrame(stage2_data)], axis=1)
val_MLP_CNN.head()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_split, y_split, stratify=y_split, train_size=0.90, random_state=0)

In [ ]:
x_train = pd.concat([x_train, x_val])

In [ ]:
x_train.shape

In [ ]:
x_val = pd.concat([x_val, x_validation])

In [ ]:
x_val.shape

In [ ]:
y_train = pd.concat([y_train, y_val])

In [ ]:
cols = y_train.columns.values.tolist()
cols
#y_train[cols] = y_train[cols].applymap(np.int32)
y_train[cols] = y_train[cols].astype(int)
y_train = y_train.values
y_train

In [ ]:
y_train.dtype

In [ ]:
y_val.isnull().values.any()

In [ ]:
y_validation = y_validation.rename(columns={"class1": "Class_0", "class2": "Class_1", 
                                            "class3": "Class_2", "class4": "Class_3", 
                                            "class5": "Class_4", "class6": "Class_5", 
                                            "class7": "Class_6", "class8": "Class_7", 
                                            "class9": "Class_8"})
y_validation.head()

In [ ]:
y_val = pd.concat([y_val, y_validation])

In [ ]:
val_cols = y_val.columns.values.tolist()
val_cols

#y_train[cols] = y_train[cols].applymap(np.int32)
y_val[val_cols] = y_val[val_cols].astype(int)
y_val = y_val.values
y_val

In [ ]:
y_val.dtype

In [ ]:
y_val.shape

In [ ]:
train_no_text.shape

In [ ]:
#x_train.iloc[:,train_no_text.shape[1]:].head()
x_train.iloc[:,:train_no_text.shape[1]].head()

In [ ]:
x_train_cnn = x_train.iloc[:,train_no_text.shape[1]:]
x_train_cnn = x_train_cnn.values

x_val_cnn = x_val.iloc[:,train_no_text.shape[1]:]
x_val_cnn = x_val_cnn.values

In [ ]:
x_train_cnn

In [ ]:
x_val_cnn

In [ ]:
x_train_fc = x_train.iloc[:,:train_no_text.shape[1]]
x_train_fc = x_train_fc.drop(['ID'], axis=1)
x_train_fc = x_train_fc.values

x_val_fc = x_val.iloc[:,:train_no_text.shape[1]]
x_val_fc = x_val_fc.drop(['ID'], axis=1)
x_val_fc = x_val_fc.values

In [ ]:
x_train_fc

In [ ]:
x_val_fc

In [ ]:
x_val_fc.shape

In [ ]:
test_cnn = test_MLP_CNN.iloc[:,test_full_no_text.shape[1]:]
test_cnn = test_cnn.values

In [ ]:
test_cnn

In [ ]:
test_fc = test_MLP_CNN.iloc[:,:test_full_no_text.shape[1]]
test_fc = test_fc.drop(['ID'], axis=1)
test_fc = test_fc.values

In [ ]:
np.where(test_fc[4] > 0)

In [ ]:
stage2_cnn = stage2_MLP_CNN.iloc[:,stage2_no_text.shape[1]:]
stage2_cnn = stage2_cnn.values

In [ ]:
stage2_cnn

In [ ]:
stage2_fc = stage2_MLP_CNN.iloc[:,:stage2_no_text.shape[1]]
stage2_fc = stage2_fc.drop(['ID'], axis=1)
stage2_fc = stage2_fc.values

In [ ]:
stage2_fc

In [ ]:
len(WORD_INDEX)

In [ ]:
SEQ_LEN

In [ ]:
embedding_matrix.shape

In [ ]:
#Glove trained embedding
embedding_layer = Embedding(input_dim=len(WORD_INDEX) + 1,
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=SEQ_LEN,
                            trainable=False)

In [ ]:
sequence_input = Input(shape=(SEQ_LEN,), dtype='int32', name='main_input')
embedded_sequences = embedding_layer(sequence_input)

conv = Conv1D(64, 3, activation='relu')(embedded_sequences)
conv = MaxPooling1D(5)(conv) #3 #add dropout after this
conv = BatchNormalization()(conv)
conv = Conv1D(128, 3, activation='relu')(conv)
conv = MaxPooling1D(5)(conv)
conv = Conv1D(256, 3, activation='relu')(conv) #remove
conv = MaxPooling1D(35)(conv) #remove
conv = Dropout(0.7)(conv)
conv = Flatten()(conv)
conv = BatchNormalization()(conv)
conv = Dropout(0.7)(conv)
conv = Dense(256, activation='relu')(conv)
conv = BatchNormalization()(conv)
conv = Dropout(0.7)(conv)
conv_out = Dense(9, activation='softmax')(conv)

#cnn = Model(inputs=sequence_input, outputs=conv_predictions)
#cnn.summary()

In [ ]:
FC_SEQ_LEN = int(x_train_fc.shape[1])
FC_SEQ_LEN

In [ ]:
auxiliary_input = Input(shape=(FC_SEQ_LEN,), name='aux_input')

x = keras.layers.concatenate([conv_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(9, activation='softmax', name='main_output')(x)

In [ ]:
model = Model(inputs=[sequence_input, auxiliary_input], outputs=[main_output])

In [ ]:
#lr=0.001

In [ ]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              loss_weights={'main_output': 1.0},
              metrics=['acc'])

In [ ]:
#train initial model with 5 epochs
#train model second time (with test predictions) with 4 epochs
####################

model.fit({'main_input': x_train_cnn, 'aux_input': x_train_fc}, {'main_output': y_train}, 
          validation_data=({'main_input': x_val_cnn, 'aux_input': x_val_fc}, {'main_output': y_val}),
          epochs=4, batch_size=128)

In [ ]:
test_preds = model.predict({'main_input': test_cnn, 'aux_input': test_fc}, batch_size=128)

In [ ]:
test_preds

In [ ]:
test_labels = np.argmax(test_preds, axis=1)
test_labels = pd.DataFrame(test_labels, columns = ['Class'])
test_labels['Class'].unique()

In [ ]:
test_labels = test_labels.append(pd.DataFrame([2,7,8], columns = ['Class']))
test_labels.head()

In [ ]:
test_labels.tail()

In [ ]:
test_labels_one_hot = keras.utils.to_categorical(test_labels['Class'])
test_labels_one_hot = pd.DataFrame(test_labels_one_hot)
test_labels_one_hot.tail()

In [ ]:
test_labels_one_hot = test_labels_one_hot.drop(test_labels_one_hot.index[[5300,5301,5302]])
test_labels_one_hot.tail()

In [ ]:
test_labels_arr = np.array(test_labels_one_hot.astype(int))
test_labels_arr

In [ ]:
sns.distplot(test_labels['Class'])

In [ ]:
test_labels_arr[3]

In [ ]:
x_train_cnn = np.vstack((x_train_cnn,test_cnn))

In [ ]:
x_train_fc = np.vstack((x_train_fc,test_fc))

In [ ]:
y_train = np.vstack((y_train, test_labels_arr))

In [ ]:
stage2_preds_4epochs = model.predict({'main_input': stage2_cnn, 'aux_input': stage2_fc}, batch_size=128)

In [ ]:
csv_5epochs = pd.DataFrame(stage2_preds_5epochs)
csv_5epochs.to_csv('stage2_preds_5epochs.csv', sep=',')
csv_5epochs.head()

In [ ]:
stage2_5eps_labels = np.argmax(np.asarray(csv_5epochs), axis=1)
stage2_5eps_labels = pd.DataFrame(stage2_5eps_labels, columns = ['Class'])
stage2_5eps_labels['Class'] = stage2_5eps_labels['Class'].astype(str)
#test_labels['Class'].unique()
stage2_5eps_labels = pd.get_dummies(stage2_5eps_labels)
stage2_5eps_labels.head()

In [ ]:
stage2_5eps_labels.to_csv('stage2_5eps_labels.csv', sep=',')

In [ ]:
csv_4epochs = pd.DataFrame(stage2_preds_4epochs)
csv_4epochs.to_csv('stage2_preds_4epochs.csv', sep=',')
csv_4epochs.head()

In [ ]:
stage2_4ep_labels = np.argmax(np.asarray(csv_4epochs), axis=1)
stage2_4ep_labels = pd.DataFrame(stage2_4ep_labels, columns = ['Class'])
stage2_4ep_labels['Class'] = stage2_4ep_labels['Class'].astype(str)
#test_labels['Class'].unique()
stage2_4ep_labels = pd.get_dummies(stage2_4ep_labels)
stage2_4ep_labels.head()